In [1]:
import ads


In [2]:
ads.__version__


'2.9.0'

## Aula  Lectura de la base de datos

In [1]:
from ads.dataset.factory import DatasetFactory

In [3]:
ds = DatasetFactory.open("data/datos_salud_final.csv")

loop1:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
ds.head()

In [ ]:
type(ds)

In [ ]:
ds.columns

In [ ]:
ds = DatasetFactory.open("datos_salud_final.csv", target="riesgo_eac_decada")
ds.head()

In [ ]:
type(ds)

In [ ]:
ds.target.show_in_notebook()

## 1.4 Creando nuestro primer modelo

In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
train, test = ds.train_test_split()

In [ ]:
train.X

In [ ]:
test.X

In [ ]:
train.y

In [ ]:
test.y

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
primer_modelo = DecisionTreeClassifier()


In [ ]:
primer_modelo.fit(train.X, train.y)

In [ ]:
primer_modelo.score(test.X, test.y)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
matriz = ConfusionMatrixDisplay.from_predictions(estimetor=primer_modelo, X = test.X, y = test.y)

### 1.5 Mejorando el primer modelo

In [ ]:
modelo_profundo = DecisionTreeClassifier(max_depth=100)

In [ ]:
modelo_profundo.fit(train.X, train.y)

In [ ]:
modelo_profundo.score(test.X, test.y)

In [ ]:
matriz = ConfusionMatrixDisplay.from_estimator(estimetor=modelo_profundo, X = test.X, y = test.y)

### 1.6 Selección de Features

In [ ]:
atributos = primer_modelo.feature_importances_
atributos

In [ ]:
import pandas as pd

pd.Series(atributos, index=train.X.columns).sort_values()

In [ ]:
train.X.drop(columns="diabetes", axis=1, inplace=True)

In [ ]:
train.X.columns

In [ ]:
primer_modelo.fit(train.X, train.y)

In [ ]:
primer_modelo.score(test.X, test.y)

In [ ]:
matriz = ConfusionMatrixDisplay.from_estimator(estimetor=primer_modelo, X = test.X, y = test.y)

### 2.1 Explorando AutoML de Oracle

In [2]:
from ads.common.data import ADSData

In [ ]:
ads_data = ADSData.build(ds.to_pandas(),"riesgo_eac_decada")
type(ads_data)

In [ ]:
from ads.automl.driver import AutoML

In [ ]:
oracle_automl = AutoML(ads_data)

In [ ]:
modelo_automl, baseline = oracle_automl.train(score_metric="accuracy", model_list=["DecisionTreeClassifier", 
                                                                                   "SVC", 
                                                                                   "logisticRegression",
                                                                                   "RandomForestClassifier"],
                                                                                   time_budget=150)

In [ ]:
oracle_automl.print_trials(max_row=20, sort_column="Mean Validation Score")

In [ ]:
oracle_automl.visualize_algorithm_selection_trials()

### 2.3 Features y modelo seleccionado

In [ ]:
modelo_automl.show_in_notebook()

In [ ]:
oracle_automl.visualize_algorithm_selection_trials()

### 3.1 ADS Evaluator

In [ ]:
modelo_automl.feature_names()

In [ ]:
train, test = ds[['sexo', 'edad', 'fumador', 'cigarrillos_por_dia',
       'uso_medicamento_presion', 'acv', 'hipertension', 'colesterol_total',
       'presion_arterial_sistolica', 'presion_arterial_diastolica', 'imc',
       'frecuencia_cardiaca', 'glicemia', 'categoria_de_fumador', "riesgo_eaac_decada"]].train_test_split()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
modelo_automl.summary()

In [ ]:
parametros = {"se usa el dicionario"}

In [ ]:
mejor_modelo = RandomForestClassifier(**parametros)

In [ ]:
primer_modelo = DecisionTreeClassifier(max_depth=100)

In [ ]:
mejor_modelo.fit(train.X, train.y)
primer_modelo.fit(train.X, train.y)

In [ ]:
from ads.common.model import ADSModel

In [ ]:
ads_mejor_modelo = ADSModel.from_estimator(mejor_modelo, name="Mejor Modelo")
ads_primer_modelo = ADSModel.from_estimator(primer_modelo, name="Primer Modelo")

In [ ]:
from ads.evaluations.evaluator import ADSEvaluator

In [ ]:
evaluator = ADSEvaluator(test_data=test, models=[ads_mejor_modelo, ads_primer_modelo], training_data=train)

### 3.2 Analizando las metricas

In [ ]:
evaluator.show_in_notebook()

#### 3.3 Evaluando más métricas

In [ ]:
evaluator.metrics

### 3.4 Overfitting

In [ ]:
primer_modelo = DecisionTreeClassifier(max_depth=5)
primer_modelo.fit(train.X, train.y)
ads_primer_modelo = ADSModel.from_estimator(primer_modelo, name="Primer Modelo")

In [ ]:
evaluator = ADSEvaluator(test_data=test, models=[ads_mejor_modelo, ads_primer_modelo], training_data=train)

In [ ]:
evaluator.metrics

### 4.1 Model Explainability

In [ ]:
from ads.explanations.explainer import ADSExplainer

In [ ]:
explainer = ADSExplainer(test, ads_mejor_modelo, train)

In [ ]:
explainer_global = explainer.global_explanation()

In [ ]:
importance = explainer_global.compute_feature_importance()

In [ ]:
importance.show_in_notebook("detailed")

### 4.2 Valores de Features x Clasificación

In [ ]:
pdp_edad = explainer_global.compute_partial_dependence("edad")

In [ ]:
pdp_presion = explainer_global.compute_partial_dependence("presion_arterial_diastolica")

In [ ]:
pdp_presion.show_in_notebook()

In [ ]:
pdp_edad.show_in_notebook()

### 4.3 Explicando decisión individual

In [ ]:
local_explainer = explainer.local_explanation()

In [ ]:
test.X.iloc[[13]]

In [ ]:
test.y.iloc[[13]]

In [ ]:
explanation = local_explainer.explain(test.X.iloc[[13]], test.y.iloc[[13]])

In [ ]:
explanation.show_in_notebook()

### 5.1 Creando nuestro artefacto

In [ ]:
path_artifac = "/home/datascience/modelo"

In [ ]:
artifact_model = ads_mejor_modelo.prepare(path_artifac, data_sample=test, data_science_env=True, force_overwrite=True)

### 5.2 Catálogo de modelos

In [ ]:
artifact_model.save(display_name = "Modelo de Clasificación de EAC")

### 5.3 Importando el modelo del catálogo

In [2]:
from ads.catalog.model import ModelCatalog

In [3]:
mc = ModelCatalog()

  mc = ModelCatalog()



ERROR - Exception
Traceback (most recent call last):
  File "/home/celis/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_722/1891029031.py", line 1, in <cell line: 0>
    mc = ModelCatalog()
         ^^^^^^^^^^^^^^
  File "/home/celis/.local/lib/python3.11/site-packages/ads/common/decorator/deprecate.py", line 65, in wrapper
    return target(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/celis/.local/lib/python3.11/site-packages/ads/catalog/model.py", line 658, in __init__
    raise ValueError("compartment_id needs to be specified.")
ValueError: compartment_id needs to be specified.
ValueError: compartment_id needs to be specified.

In [ ]:
mc_list = mc.list_models()

In [ ]:
mc_list[0]

### 5.4 API del modelo 

In [ ]:
model_id = mc_list[0].id

In [4]:
from ads.model.deployment import ModelDeployer

In [5]:
deployer = ModelDeployer()

ERROR - Exception
Traceback (most recent call last):
  File "/home/celis/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_722/1784797373.py", line 1, in <cell line: 0>
    deployer = ModelDeployer()
               ^^^^^^^^^^^^^^^
  File "/home/celis/.local/lib/python3.11/site-packages/ads/model/deployment/model_deployer.py", line 122, in __init__
    self.client_manager = OCIClientManager()
                          ^^^^^^^^^^^^^^^^^^
  File "/home/celis/.local/lib/python3.11/site-packages/ads/model/deployment/common/utils.py", line 193, in __init__
    config = auth.default_signer()
             ^^^^^^^^^^^^^^^^^^^^^
  File "/home/celis/.local/lib/python3.11/site-packages/ads/common/auth.py", line 508, in default_signer
    return signer_generator(signer_args).create_signer()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/celis/.local/lib/py

In [6]:
import os

In [ ]:
deployer.deploy(model_id=model_id,
                display_name="API del modelo de clasificación de EAC",
                instance_shape="VM.Standard2.1",
                instance_count=1,
                project_id=os.environ["PROJECT_OCID"],
                compartment_id=os.environ["TENANCY_OCID"])

In [ ]:
bady = test.X.iloc[[5,-1],:].to_dict()

In [7]:
# The OCI SDK must be installed for this example to function properly.
# Installation instructions can be found here: https://docs.oracle.com/en-us/iaas/Content/API/SDKDocs/pythonsdk.htm

import requests
import oci
from oci.signer import Signer

config = oci.config.from_file("~/.oci/config") # replace with the location of your oci config file
auth = Signer(
  tenancy=config['tenancy'],
  user=config['user'],
  fingerprint=config['fingerprint'],
  private_key_file_location=config['key_file'],
  pass_phrase=config['pass_phrase'])

endpoint = 'https://modeldeployment.sa-saopaulo-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.sa-saopaulo-1.amaaaaaaqzwhveqabez4udtjlddt4elxxr4fxhw44ded62vrlwyxjxnry6vq/predict'
#body = {} # payload goes here

requests.post(endpoint, json=body, auth=auth).json()

ERROR - Exception
Traceback (most recent call last):
  File "/home/celis/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_722/1701978168.py", line 8, in <cell line: 0>
    config = oci.config.from_file("~/.oci/config") # replace with the location of your oci config file
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/celis/.local/lib/python3.11/site-packages/oci/config.py", line 107, in from_file
    raise ConfigFileNotFound("Could not find config file at {}, please follow the instructions in the link to setup the config file https://docs.cloud.oracle.com/en-us/iaas/Content/API/Concepts/sdkconfig.htm".format(expanded_file_location))
oci.exceptions.ConfigFileNotFound: Could not find config file at /home/celis/.oci/config, please follow the instructions in the link to setup the config file https://docs.cloud.oracle.com/en-us/iaas/Content/API/Concepts/